# Olfactory Nupic Spatial Pooling with Variable Learning Rates

In [1]:
import csv
import datetime
import numpy 
import os
import yaml
import seaborn as sns
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt


from nupic.algorithms.sdr_classifier_factory import SDRClassifierFactory
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory
#import date encoder
from nupic.encoders.date import DateEncoder
#import RandomDistributedScalarEncoder
from nupic.encoders.random_distributed_scalar import \
  RandomDistributedScalarEncoder
#import normal ScalarEncoder
from nupic.encoders import ScalarEncoder
#category encoder
from nupic.encoders.category import CategoryEncoder

# Model Parameters

In [2]:
# Model Params!
MODEL_PARAMS = {
    # Type of model that the rest of these parameters apply to.
    'model': "Classification",

    # Version that specifies the format of the config.
    'version': 1,

    # Intermediate variables used to compute fields in modelParams and also
    # referenced from the control section.
    'aggregationInfo': {   'days': 0,
        'fields': [('chem1', 'sum')],
        'hours': 1,
        'microseconds': 0,
        'milliseconds': 0,
        'minutes': 0,
        'months': 0,
        'seconds': 0,
        'weeks': 0,
        'years': 0},

    'predictAheadTime': None,

    # Model parameter dictionary.
    'modelParams': {
        # The type of inference that this model will perform
        'inferenceType': 'TemporalAnomaly',

        'sensorParams': {
            # Sensor diagnostic output verbosity control;
            # if > 0: sensor region will print out on screen what it's sensing
            # at each step 0: silent; >=1: some info; >=2: more info;
            # >=3: even more info (see compute() in py/regions/RecordSensor.py)
            'verbosity' : 0,

            # Include the encoders we use
            'encoders': {
                u'timestamp_timeOfDay': {
                    'fieldname': u'timestamp',
                    'name': u'timestamp_timeOfDay',
                    'timeOfDay': (21, 0.5),
                    'type': 'DateEncoder'},
                u'timestamp_dayOfWeek': None,
                u'timestamp_weekend': { 
                    'fieldname': u'timestamp',
                    'name': u' timestamp_weekend',
                    'weekend': 21,
                    'type': 'DateEncoder'}, 
                u'chem1':{
                    'clipInput': True,
                    'fieldname': u'chem1',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'chem1':{
                    'clipInput': True,
                    'fieldname': u'chem1',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'chem2':{
                    'clipInput': True,
                    'fieldname': u'chem2',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'chem3':{
                    'clipInput': True,
                    'fieldname': u'chem3',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'chem4':{
                    'clipInput': True,
                    'fieldname': u'chem4',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'scent':{'fieldname': u'scent',
                    'categoryList':['lemon','flower','dog','sugar'],
                    'forced':False,
                    'name': u'chem',
                    'type': 'CategoryEncoder',
                    'w': 21,
                    'resolution':0.88},},
            # CategoryEncoder(self, w, categoryList, name='category', verbosity=0, forced=False)
            # A dictionary specifying the period for automatically-generated
            # resets from a RecordSensor;
            #
            # None = disable automatically-generated resets (also disabled if
            # all of the specified values evaluate to 0).
            # Valid keys is the desired combination of the following:
            #   days, hours, minutes, seconds, milliseconds, microseconds, weeks
            #
            # Example for 1.5 days: sensorAutoReset = dict(days=1,hours=12),
            #
            # (value generated from SENSOR_AUTO_RESET)
            'sensorAutoReset' : None,
        },

        'spEnable': True,

        'spParams': {
            # SP diagnostic output verbosity control;
            # 0: silent; >=1: some info; >=2: more info;
            'spVerbosity' : 0,

            # Spatial Pooler implementation selector, see getSPClass
            # in py/regions/SPRegion.py for details
            # 'py' (default), 'cpp' (speed optimized, new)
            'spatialImp' : 'cpp',

            'globalInhibition': 1,

            # Number of cell columns in the cortical region (same number for
            # SP and TM)
            # (see also tpNCellsPerCol)
            'columnCount': 2650,

            'inputWidth': 0,

            # SP inhibition control (absolute value);
            # Maximum number of active columns in the SP region's output (when
            # there are more, the weaker ones are suppressed)
            'numActiveColumnsPerInhArea': 40,

            'seed': 1956,

            # potentialPct
            # What percent of the columns's receptive field is available
            # for potential synapses. At initialization time, we will
            # choose potentialPct * (2*potentialRadius+1)^2
            'potentialPct': 0.5,
            'localAreaDensity': -1.0,
            'boostStrength': 0.0,

            # The default connected threshold. Any synapse whose
            # permanence value is above the connected threshold is
            # a "connected synapse", meaning it can contribute to the
            # cell's firing. Typical value is 0.10. Cells whose activity
            # level before inhibition falls below minDutyCycleBeforeInh
            # will have their own internal synPermConnectedCell
            # threshold set below this default value.
            # (This concept applies to both SP and TM and so 'cells'
            # is correct here as opposed to 'columns')
            'synPermConnected': 0.1,

            'synPermActiveInc': 0.1,

            'synPermInactiveDec': 0.005,
        },

        # Controls whether TM is enabled or disabled;
        # TM is necessary for making temporal predictions, such as predicting
        # the next inputs.  Without TP, the model is only capable of
        # reconstructing missing sensor inputs (via SP).
        'tmEnable' : False,

        'tmParams': {
            # TM diagnostic output verbosity control;
            # 0: silent; [1..6]: increasing levels of verbosity
            # (see verbosity in nupic/trunk/py/nupic/research/TP.py and BacktrackingTMCPP.py)
            'verbosity': 0,

            # Number of cell columns in the cortical region (same number for
            # SP and TM)
            # (see also tpNCellsPerCol)
            'columnCount': 2650,

            # The number of cells (i.e., states), allocated per column.
            'cellsPerColumn': 32,

            'inputWidth': 2048,

            'seed': 1960,

            # Temporal Pooler implementation selector (see _getTPClass in
            # CLARegion.py).
            'temporalImp': 'cpp',

            # New Synapse formation count
            # NOTE: If None, use spNumActivePerInhArea
            #
            # TODO: need better explanation
            'newSynapseCount': 20,

            # Maximum number of synapses per segment
            #  > 0 for fixed-size CLA
            # -1 for non-fixed-size CLA
            #
            # TODO: for Ron: once the appropriate value is placed in TP
            # constructor, see if we should eliminate this parameter from
            # description.py.
            'maxSynapsesPerSegment': 32,

            # Maximum number of segments per cell
            #  > 0 for fixed-size CLA
            # -1 for non-fixed-size CLA
            #
            # TODO: for Ron: once the appropriate value is placed in TP
            # constructor, see if we should eliminate this parameter from
            # description.py.
            'maxSegmentsPerCell': 128,

            # Initial Permanence
            # TODO: need better explanation
            'initialPerm': 0.21,

            # Permanence Increment
            'permanenceInc': 0.1,

            # Permanence Decrement
            # If set to None, will automatically default to tpPermanenceInc
            # value.
            'permanenceDec' : 0.1,

            'globalDecay': 0.0,

            'maxAge': 0,

            # Minimum number of active synapses for a segment to be considered
            # during search for the best-matching segments.
            # None=use default
            # Replaces: tpMinThreshold
            'minThreshold': 9,

            # Segment activation threshold.
            # A segment is active if it has >= tpSegmentActivationThreshold
            # connected synapses that are active due to infActiveState
            # None=use default
            # Replaces: tpActivationThreshold
            'activationThreshold': 12,

            'outputType': 'normal',

            # "Pay Attention Mode" length. This tells the TM how many new
            # elements to append to the end of a learned sequence at a time.
            # Smaller values are better for datasets with short sequences,
            # higher values are better for datasets with long sequences.
            'pamLength': 1,
        },

        'clParams': {
            'regionName' : 'SDRClassifierRegion',

            # Classifier diagnostic output verbosity control;
            # 0: silent; [1..6]: increasing levels of verbosity
            'verbosity' : 0,

            # This controls how fast the classifier learns/forgets. Higher values
            # make it adapt faster and forget older patterns faster.
            'alpha': 0.005,

            # This is set after the call to updateConfigFromSubConfig and is
            # computed from the aggregationInfo and predictAheadTime.
            'steps': '1',

            'implementation': 'cpp',
        },

        'anomalyParams': {
            u'anomalyCacheRecords': None,
            u'autoDetectThreshold': None,
            u'autoDetectWaitRecords': 2184
        },

        'trainSPNetOnlyIfRequested': False,
    },
}

# Encoding Data

In [22]:
_NUM_RECORDS = 16000
_EXAMPLE_DIR = os.path.dirname(os.path.abspath('/Users/adamcasper/nupic/examples'))
_INPUT_FILE_PATH = "data50_with_dopa.csv"

In [23]:
#feeding model params to different encoders
modelParams = MODEL_PARAMS["modelParams"]
enParams = modelParams["sensorParams"]["encoders"]
spParams = modelParams["spParams"]
tmParams = modelParams["tmParams"]


#encodes time of day
#timeOfDayEncoder = DateEncoder(
#timeOfDay=enParams["timestamp_timeOfDay"]["timeOfDay"])
#encodes weekend
#weekendEncoder = DateEncoder(
#weekend=enParams["timestamp_weekend"]["weekend"])
#scalar encoder for chem1
scalarEncoder = RandomDistributedScalarEncoder(
enParams["chem1"]['resolution'])
#category Encoder
categoryEncoder = CategoryEncoder(w=enParams['scent']['w'],categoryList=enParams['scent']['categoryList'],\
                                      forced=enParams['scent']['forced'])



In [24]:
#encode width for full input encoding 
one_scalar = scalarEncoder.getWidth()
scent_width = 50*one_scalar
    
encodingWidth = (scent_width
                 + categoryEncoder.getWidth())

In [25]:
sp = SpatialPooler(
  # How large the input encoding will be.
  inputDimensions=(encodingWidth),
  # How many mini-columns will be in the Spatial Pooler.
  columnDimensions=(2025),
  # What percent of the columns's receptive field is available for potential
  # synapses?
  potentialPct=0.85,
  # This means that the input space has no topology.
  globalInhibition=True,
  localAreaDensity=-1.0,
  # Roughly 2%, giving that there is only one inhibition area because we have
  # turned on globalInhibition (40 / 2048 = 0.0195)
  numActiveColumnsPerInhArea=40.0,
  # How quickly synapses grow and degrade.
  synPermInactiveDec=0.005,
  synPermActiveInc=0.04,
  synPermConnected=0.1,
  # boostStrength controls the strength of boosting. Boosting encourages
  # efficient usage of SP columns.
  boostStrength=3.0,
  # Random number generator seed.
  seed=1956,
  # Determines if inputs at the beginning and end of an input dimension should
  # be considered neighbors when mapping columns to inputs.
  wrapAround=False
)

In [26]:
ArrayOfColumns = numpy.zeros((2025), dtype="int")
activeDutyCycles = numpy.zeros((2025), dtype="float")




In [27]:
#create SDR classifier instance
classifier = SDRClassifierFactory.create(steps=[0])

In [28]:
from nupic.algorithms.sdr_classifier import SDRClassifier


c = SDRClassifier(steps=[0], alpha=0.1, actValueAlpha=0.1, verbosity=0)

numpy.set_printoptions(threshold=numpy.inf)


In [31]:

c = SDRClassifier(steps=[0], alpha=0.1, actValueAlpha=0.1, verbosity=0)

numRecords = 1000
inputFile = open(_INPUT_FILE_PATH, "rb")
reader = csv.reader(inputFile)
# skip header rows
reader.next()
reader.next()
reader.next()

counter = 0
for record in reader:
   
    counter += 1
    if counter >= numRecords: break
            

    # Convert data string into Python date object.
    #dateString = datetime.datetime.strptime(record[1], "%m/%d/%y %H:%M")
    scentString = record[0]
    # Convert data value string into float.
    scent = categoryEncoder.encode(record[0])
    chem1 = float(record[1])
    chem2 = float(record[2])
    chem3 = float(record[3])
    chem4 = float(record[4])
    chem5 = float(record[5])
    chem6 = float(record[6])
    chem7 = float(record[7])
    chem8 = float(record[8])
    chem9 = float(record[9])
    chem10 = float(record[10])
    chem11 = float(record[11])
    chem12 = float(record[12])
    chem13 = float(record[13])
    chem14 = float(record[14])
    chem15 = float(record[15])
    chem16 = float(record[16])
    chem17 = float(record[17])
    chem18 = float(record[18])
    chem19 = float(record[19])
    chem20 = float(record[20])
    chem21 = float(record[21])
    chem22 = float(record[22])
    chem23 = float(record[23])
    chem24 = float(record[24])
    chem25 = float(record[25])
    chem26 = float(record[26])
    chem27 = float(record[27])
    chem28 = float(record[28])
    chem29 = float(record[29])
    chem30 = float(record[30])
    chem31 = float(record[31])
    chem32 = float(record[32])
    chem33 = float(record[33])
    chem34 = float(record[34])
    chem35 = float(record[35])
    chem36 = float(record[36])
    chem37 = float(record[37])
    chem38 = float(record[38])
    chem39 = float(record[39])
    chem40 = float(record[40])
    chem41 = float(record[41])
    chem42 = float(record[42])
    chem43 = float(record[43])
    chem44 = float(record[44])
    chem45 = float(record[45])
    chem46 = float(record[46])
    chem47 = float(record[47])
    chem48 = float(record[48])
    chem49 = float(record[49])
    chem50 = float(record[50])
    dopa = int(record[51])
        

        # To encode, we need to provide zero-filled numpy arrays for the encoders
        # to populate.
        #timeOfDayBits = numpy.zeros(timeOfDayEncoder.getWidth())
        #weekendBits = numpy.zeros(weekendEncoder.getWidth())
    scentBits = numpy.zeros(categoryEncoder.getWidth())
    chem1Bits = numpy.zeros(scalarEncoder.getWidth())
    chem2Bits = numpy.zeros(scalarEncoder.getWidth())
    chem3Bits = numpy.zeros(scalarEncoder.getWidth())
    chem4Bits = numpy.zeros(scalarEncoder.getWidth())
    chem5Bits = numpy.zeros(scalarEncoder.getWidth())
    chem6Bits = numpy.zeros(scalarEncoder.getWidth())
    chem7Bits = numpy.zeros(scalarEncoder.getWidth())
    chem8Bits = numpy.zeros(scalarEncoder.getWidth())
    chem9Bits = numpy.zeros(scalarEncoder.getWidth())
    chem10Bits = numpy.zeros(scalarEncoder.getWidth())
    chem11Bits = numpy.zeros(scalarEncoder.getWidth())
    chem12Bits = numpy.zeros(scalarEncoder.getWidth())
    chem13Bits = numpy.zeros(scalarEncoder.getWidth())
    chem14Bits = numpy.zeros(scalarEncoder.getWidth())
    chem15Bits = numpy.zeros(scalarEncoder.getWidth())
    chem16Bits = numpy.zeros(scalarEncoder.getWidth())
    chem17Bits = numpy.zeros(scalarEncoder.getWidth())
    chem18Bits = numpy.zeros(scalarEncoder.getWidth())
    chem19Bits = numpy.zeros(scalarEncoder.getWidth())
    chem20Bits = numpy.zeros(scalarEncoder.getWidth())
    chem21Bits = numpy.zeros(scalarEncoder.getWidth())
    chem22Bits = numpy.zeros(scalarEncoder.getWidth())
    chem23Bits = numpy.zeros(scalarEncoder.getWidth())
    chem24Bits = numpy.zeros(scalarEncoder.getWidth())
    chem25Bits = numpy.zeros(scalarEncoder.getWidth())
    chem26Bits = numpy.zeros(scalarEncoder.getWidth())
    chem27Bits = numpy.zeros(scalarEncoder.getWidth())
    chem28Bits = numpy.zeros(scalarEncoder.getWidth())
    chem29Bits = numpy.zeros(scalarEncoder.getWidth())
    chem30Bits = numpy.zeros(scalarEncoder.getWidth())
    chem31Bits = numpy.zeros(scalarEncoder.getWidth())
    chem32Bits = numpy.zeros(scalarEncoder.getWidth())
    chem33Bits = numpy.zeros(scalarEncoder.getWidth())
    chem34Bits = numpy.zeros(scalarEncoder.getWidth())
    chem35Bits = numpy.zeros(scalarEncoder.getWidth())
    chem36Bits = numpy.zeros(scalarEncoder.getWidth())
    chem37Bits = numpy.zeros(scalarEncoder.getWidth())
    chem38Bits = numpy.zeros(scalarEncoder.getWidth())
    chem39Bits = numpy.zeros(scalarEncoder.getWidth())
    chem40Bits = numpy.zeros(scalarEncoder.getWidth())
    chem41Bits = numpy.zeros(scalarEncoder.getWidth())
    chem42Bits = numpy.zeros(scalarEncoder.getWidth())
    chem43Bits = numpy.zeros(scalarEncoder.getWidth())
    chem44Bits = numpy.zeros(scalarEncoder.getWidth())
    chem45Bits = numpy.zeros(scalarEncoder.getWidth())
    chem46Bits = numpy.zeros(scalarEncoder.getWidth())
    chem47Bits = numpy.zeros(scalarEncoder.getWidth())
    chem48Bits = numpy.zeros(scalarEncoder.getWidth())
    chem49Bits = numpy.zeros(scalarEncoder.getWidth())
    chem50Bits = numpy.zeros(scalarEncoder.getWidth())
    
    
    dopa = int(record[51])
    
    
    # Now we call the encoders to create bit representations for each value.
    #timeOfDayEncoder.encodeIntoArray(dateString, timeOfDayBits)
    #weekendEncoder.encodeIntoArray(dateString, weekendBits)
    scentBits = numpy.zeros(categoryEncoder.getWidth())
    chem1Bits = numpy.zeros(scalarEncoder.getWidth())
    chem2Bits = numpy.zeros(scalarEncoder.getWidth())
    chem3Bits = numpy.zeros(scalarEncoder.getWidth())
    chem4Bits = numpy.zeros(scalarEncoder.getWidth())
    chem5Bits = numpy.zeros(scalarEncoder.getWidth())
    chem6Bits = numpy.zeros(scalarEncoder.getWidth())
    chem7Bits = numpy.zeros(scalarEncoder.getWidth())
    chem8Bits = numpy.zeros(scalarEncoder.getWidth())
    chem9Bits = numpy.zeros(scalarEncoder.getWidth())
    chem10Bits = numpy.zeros(scalarEncoder.getWidth())
    chem11Bits = numpy.zeros(scalarEncoder.getWidth())
    chem12Bits = numpy.zeros(scalarEncoder.getWidth())
    chem13Bits = numpy.zeros(scalarEncoder.getWidth())
    chem14Bits = numpy.zeros(scalarEncoder.getWidth())
    chem15Bits = numpy.zeros(scalarEncoder.getWidth())
    chem16Bits = numpy.zeros(scalarEncoder.getWidth())
    chem17Bits = numpy.zeros(scalarEncoder.getWidth())
    chem18Bits = numpy.zeros(scalarEncoder.getWidth())
    chem19Bits = numpy.zeros(scalarEncoder.getWidth())
    chem20Bits = numpy.zeros(scalarEncoder.getWidth())
    chem21Bits = numpy.zeros(scalarEncoder.getWidth())
    chem22Bits = numpy.zeros(scalarEncoder.getWidth())
    chem23Bits = numpy.zeros(scalarEncoder.getWidth())
    chem24Bits = numpy.zeros(scalarEncoder.getWidth())
    chem25Bits = numpy.zeros(scalarEncoder.getWidth())
    chem26Bits = numpy.zeros(scalarEncoder.getWidth())
    chem27Bits = numpy.zeros(scalarEncoder.getWidth())
    chem28Bits = numpy.zeros(scalarEncoder.getWidth())
    chem29Bits = numpy.zeros(scalarEncoder.getWidth())
    chem30Bits = numpy.zeros(scalarEncoder.getWidth())
    chem31Bits = numpy.zeros(scalarEncoder.getWidth())
    chem32Bits = numpy.zeros(scalarEncoder.getWidth())
    chem33Bits = numpy.zeros(scalarEncoder.getWidth())
    chem34Bits = numpy.zeros(scalarEncoder.getWidth())
    chem35Bits = numpy.zeros(scalarEncoder.getWidth())
    chem36Bits = numpy.zeros(scalarEncoder.getWidth())
    chem37Bits = numpy.zeros(scalarEncoder.getWidth())
    chem38Bits = numpy.zeros(scalarEncoder.getWidth())
    chem39Bits = numpy.zeros(scalarEncoder.getWidth())
    chem40Bits = numpy.zeros(scalarEncoder.getWidth())
    chem41Bits = numpy.zeros(scalarEncoder.getWidth())
    chem42Bits = numpy.zeros(scalarEncoder.getWidth())
    chem43Bits = numpy.zeros(scalarEncoder.getWidth())
    chem44Bits = numpy.zeros(scalarEncoder.getWidth())
    chem45Bits = numpy.zeros(scalarEncoder.getWidth())
    chem46Bits = numpy.zeros(scalarEncoder.getWidth())
    chem47Bits = numpy.zeros(scalarEncoder.getWidth())
    chem48Bits = numpy.zeros(scalarEncoder.getWidth())
    chem49Bits = numpy.zeros(scalarEncoder.getWidth())
    chem50Bits = numpy.zeros(scalarEncoder.getWidth())
        
    
    
    categoryEncoder.encodeIntoArray(scentString,scentBits)
    scalarEncoder.encodeIntoArray(chem1, chem1Bits)
    scalarEncoder.encodeIntoArray(chem2, chem2Bits)
    scalarEncoder.encodeIntoArray(chem3, chem3Bits)
    scalarEncoder.encodeIntoArray(chem4, chem4Bits)
    scalarEncoder.encodeIntoArray(chem5, chem5Bits)
    scalarEncoder.encodeIntoArray(chem6, chem6Bits)
    scalarEncoder.encodeIntoArray(chem7, chem7Bits)
    scalarEncoder.encodeIntoArray(chem8, chem8Bits)
    scalarEncoder.encodeIntoArray(chem9, chem9Bits)
    scalarEncoder.encodeIntoArray(chem10, chem10Bits)
    scalarEncoder.encodeIntoArray(chem11, chem11Bits)
    scalarEncoder.encodeIntoArray(chem12, chem12Bits)
    scalarEncoder.encodeIntoArray(chem13, chem13Bits)
    scalarEncoder.encodeIntoArray(chem14, chem14Bits)
    scalarEncoder.encodeIntoArray(chem15, chem15Bits)
    scalarEncoder.encodeIntoArray(chem16, chem16Bits)
    scalarEncoder.encodeIntoArray(chem17, chem17Bits)
    scalarEncoder.encodeIntoArray(chem18, chem18Bits)
    scalarEncoder.encodeIntoArray(chem19, chem19Bits)
    scalarEncoder.encodeIntoArray(chem20, chem20Bits)
    scalarEncoder.encodeIntoArray(chem21, chem21Bits)
    scalarEncoder.encodeIntoArray(chem22, chem22Bits)
    scalarEncoder.encodeIntoArray(chem23, chem23Bits)
    scalarEncoder.encodeIntoArray(chem24, chem24Bits)
    scalarEncoder.encodeIntoArray(chem25, chem25Bits)
    scalarEncoder.encodeIntoArray(chem26, chem26Bits)
    scalarEncoder.encodeIntoArray(chem27, chem27Bits)
    scalarEncoder.encodeIntoArray(chem28, chem28Bits)
    scalarEncoder.encodeIntoArray(chem29, chem29Bits)
    scalarEncoder.encodeIntoArray(chem30, chem30Bits)
    scalarEncoder.encodeIntoArray(chem31, chem31Bits)
    scalarEncoder.encodeIntoArray(chem32, chem32Bits)
    scalarEncoder.encodeIntoArray(chem33, chem33Bits)
    scalarEncoder.encodeIntoArray(chem34, chem34Bits)
    scalarEncoder.encodeIntoArray(chem35, chem35Bits)
    scalarEncoder.encodeIntoArray(chem36, chem36Bits)
    scalarEncoder.encodeIntoArray(chem37, chem37Bits)
    scalarEncoder.encodeIntoArray(chem38, chem38Bits)
    scalarEncoder.encodeIntoArray(chem39, chem39Bits)
    scalarEncoder.encodeIntoArray(chem40, chem40Bits)
    scalarEncoder.encodeIntoArray(chem41, chem41Bits)
    scalarEncoder.encodeIntoArray(chem42, chem42Bits)
    scalarEncoder.encodeIntoArray(chem43, chem43Bits)
    scalarEncoder.encodeIntoArray(chem44, chem44Bits)
    scalarEncoder.encodeIntoArray(chem45, chem45Bits)
    scalarEncoder.encodeIntoArray(chem46, chem46Bits)
    scalarEncoder.encodeIntoArray(chem47, chem47Bits)
    scalarEncoder.encodeIntoArray(chem48, chem48Bits)
    scalarEncoder.encodeIntoArray(chem49, chem49Bits)
    scalarEncoder.encodeIntoArray(chem50, chem50Bits)
        
    
     # Concatenate all these encodings into one large encoding for Spatial
        # Pooling.
            
    encoding = numpy.concatenate(
        [scentBits, chem1Bits, chem2Bits, chem3Bits, chem4Bits,\
        chem5Bits, chem6Bits, chem7Bits, chem8Bits, chem9Bits,\
        chem10Bits, chem11Bits, chem12Bits, chem13Bits, chem14Bits, chem15Bits, chem16Bits, chem17Bits,\
        chem18Bits, chem19Bits, chem20Bits, chem21Bits, chem22Bits, chem23Bits, chem24Bits, chem25Bits,\
        chem26Bits, chem27Bits, chem28Bits, chem29Bits, chem30Bits, chem31Bits, chem32Bits, chem33Bits,\
        chem34Bits, chem35Bits, chem36Bits, chem37Bits, chem38Bits, chem39Bits, chem40Bits, chem41Bits,\
        chem42Bits, chem43Bits, chem44Bits, chem45Bits, chem46Bits, chem47Bits, chem48Bits, chem49Bits, chem50Bits])
    
    activeColumnIndices = numpy.nonzero(encoding)[0]
    
    
     # Get the bucket info for this input value for classification.
    bucketIdx = categoryEncoder.getBucketIndices(scentString)[0]
    
    
    

     # learning
    c.compute(recordNum=counter, patternNZ=activeColumnIndices,
               classification={"bucketIdx": bucketIdx, "actValue": record[0]},
               learn=True, infer=False)
    # inference
    result = c.compute(recordNum=counter, patternNZ=activeColumnIndices,
               classification={"bucketIdx": bucketIdx, "actValue": record[0]},
                        learn=False, infer=True)

   

     # Print the  predictions for 0 steps out.
    pred_value = sorted(zip(result[0],
                             result["actualValues"]), reverse=True)[:4]
    
    print pred_value
    
    
    
    
    #for probability, value in topPredictions:
        #print "Prediction of {} has probability of {}.".format(value,
                                                              #probability*100.0)
    #print "Actual Value: {} Prediction of: {} has probability of {}.".format(record[0], pred_value, probability*100.0)
        
        
    
    

[(1.0, 'cow')]
[(1.0, 'sugar'), (3.069453002741848e-47, 'cow'), (3.069453002741848e-47, 0), (3.069453002741848e-47, 0)]
[(1.0, 'grass'), (3.0694530027446517e-47, 0), (3.0694530027446517e-47, 0), (3.0694530027446517e-47, 0)]
[(0.99999999872607115, 'malefly'), (1.5348549532515275e-11, 0), (1.5348549532515275e-11, 0), (1.5348549532515275e-11, 0)]
[(0.99999984486618887, 'apple'), (1.8649772261071381e-09, 0), (1.8649772261071381e-09, 0), (1.8649772261071381e-09, 0)]
[(0.99999957905574899, 'banana'), (5.0693647813161334e-09, 0), (5.0693647813161334e-09, 0), (5.0693647813161334e-09, 0)]
[(0.99999998595747241, 'frog'), (1.6918701029319872e-10, 0), (1.6918701029319872e-10, 0), (1.6918701029319872e-10, 0)]
[(0.99999984487063032, 'apple'), (1.8649238380276101e-09, 0), (1.8649238380276101e-09, 0), (1.8649238380276101e-09, 0)]
[(0.99999999872609135, 'malefly'), (1.5348303559821188e-11, 0), (1.5348303559821188e-11, 0), (1.5348303559821188e-11, 0)]
[(0.99999979096311697, 'doodoo'), (2.517398001918898

[(1.0, 'sweat'), (1.7684700235684139e-36, 'flower'), (2.9606883852991309e-42, 0), (2.9606883852991309e-42, 0)]
[(0.99999999999994016, 'malefly'), (2.0032411373148669e-14, 'dog'), (3.323512567778005e-15, 'flower'), (4.4967890990725879e-16, 0)]
[(1.0, 'sugar'), (3.2240168826849859e-20, 'malefly'), (4.1929148503365222e-37, 0), (4.1929148503365222e-37, 0)]
[(1.0, 'dog'), (8.7730252902543334e-35, 'lemon'), (4.9326442685637412e-36, 0), (4.9326442685637412e-36, 0)]
[(0.99999999994526045, 'socks'), (3.9506058386817604e-11, 'flower'), (2.3141524443823907e-12, 'sugar'), (1.6141585648803482e-13, 0)]
[(0.99999999999998401, 'frog'), (9.9730897067324227e-15, 'flower'), (9.0688279269341538e-17, 0), (9.0688279269341538e-17, 0)]
[(0.99999999101112447, 'banana'), (1.3376591475220845e-10, 'flower'), (1.0949219134013433e-10, 0), (1.0949219134013433e-10, 0)]
[(1.0, 'dog'), (8.7730253064665202e-35, 'lemon'), (4.932644266429153e-36, 0), (4.932644266429153e-36, 0)]
[(0.99999999999846645, 'apple'), (1.91076574

[(0.99999999999937594, 'table'), (6.3072251819625845e-14, 'flower'), (6.9866687484747767e-15, 0), (6.9866687484747767e-15, 0)]
[(1.0, 'cow'), (2.9345042379212406e-45, 0), (2.9345042379212406e-45, 0), (2.9345042379212406e-45, 0)]
[(1.0, 'chair'), (1.541511386134728e-45, 'sugar'), (1.2384559085343823e-46, 'dog'), (6.2712745825258597e-48, 0)]
[(1.0, 'flower'), (1.4849892897104076e-37, 0), (1.4849892897104076e-37, 0), (1.4849892897104076e-37, 0)]
[(0.99999999104617099, 'banana'), (1.3324948092852188e-10, 'flower'), (1.0906937969537746e-10, 0), (1.0906937969537746e-10, 0)]
[(1.0, 'cat'), (9.6327292147109167e-19, 'sugar'), (2.0307421727946588e-20, 0), (2.0307421727946588e-20, 0)]
[(1.0, 'chair'), (1.541511046448808e-45, 'sugar'), (1.2384556330462988e-46, 'dog'), (6.2712731843608934e-48, 0)]
[(1.0, 'flower'), (1.4849892906010997e-37, 0), (1.4849892906010997e-37, 0), (1.4849892906010997e-37, 0)]
[(0.99999999104617976, 'banana'), (1.3324935124672176e-10, 'flower'), (1.0906927382872465e-10, 0), 

[(1.0, 'sugar'), (3.2391870373090187e-20, 'shock'), (4.1929107240035885e-37, 0), (4.1929107240035885e-37, 0)]
[(1.0, 'cat'), (9.6044942763759447e-19, 'sugar'), (2.0247885192454522e-20, 0), (2.0247885192454522e-20, 0)]
[(0.9999999999456749, 'socks'), (3.9207047509402185e-11, 'flower'), (2.2966370900116375e-12, 'sugar'), (1.6019387707918187e-13, 0)]
[(0.9999999999456749, 'socks'), (3.9207047116652843e-11, 'flower'), (2.2966370760843172e-12, 'sugar'), (1.6019387614438596e-13, 0)]
[(0.99999999999913647, 'shock'), (8.6221780069451685e-13, 'sugar'), (1.6776516167435793e-17, 0), (1.6776516167435793e-17, 0)]
[(0.99999999999998412, 'frog'), (9.9080040091991722e-15, 'flower'), (9.0096304575386459e-17, 0), (9.0096304575386459e-17, 0)]
[(1.0, 'grass'), (4.588226685500345e-44, 'flower'), (5.0961835394251294e-46, 0), (5.0961835394251294e-46, 0)]
[(1.0, 'grass'), (4.588226685500345e-44, 'flower'), (5.0961835394251294e-46, 0), (5.0961835394251294e-46, 0)]
[(0.99999999999991984, 'doodoo'), (9.910865589

[(0.99999999999826561, 'orange'), (1.074578945920257e-12, 'flower'), (1.8930758958819596e-14, 'sugar'), (7.9998318780958264e-15, 0)]
[(1.0, 'dog'), (8.8687483318871216e-35, 'lemon'), (4.9864521158952076e-36, 0), (4.9864521158952076e-36, 0)]
[(0.99996000232718174, 'sheep'), (3.9239615453910398e-05, 'dog'), (9.4636042568645535e-09, 0), (9.4636042568645535e-09, 0)]
[(0.99999999999938116, 'table'), (6.2536932741521452e-14, 'flower'), (6.9273567167125725e-15, 0), (6.9273567167125725e-15, 0)]
[(0.99999999999999967, 'ladyfly'), (1.1957840788396949e-17, 'dog'), (6.0233988946504006e-18, 0), (6.0233988946504006e-18, 0)]
[(1.0, 'lemon'), (1.8511554978685392e-38, 0), (1.8511554978685392e-38, 0), (1.8511554978685392e-38, 0)]
[(1.0, 'chair'), (1.5279799808953462e-45, 'sugar'), (1.2170105532610308e-46, 'dog'), (6.2162134438677879e-48, 0)]
[(0.9999999999984851, 'apple'), (1.8875617646241093e-14, 0), (1.8875617646241093e-14, 0), (1.8875617646241093e-14, 0)]
[(1.0, 'cat'), (9.5417608820187246e-19, 'suga

...

[(0.99999999999827582, 'orange'), (1.0682032697244696e-12, 'flower'), (1.8818438206555468e-14, 'sugar'), (7.9523563911201907e-15, 0)]
[(1.0, 'flower'), (1.4849857206752453e-37, 0), (1.4849857206752453e-37, 0), (1.4849857206752453e-37, 0)]
[(0.99999999999998435, 'frog'), (9.7983073830838799e-15, 'flower'), (8.909857274202496e-17, 0), (8.909857274202496e-17, 0)]
[(0.99999999994637223, 'socks'), (3.8703806510422511e-11, 'flower'), (2.2671585713684665e-12, 'sugar'), (1.581372554506401e-13, 0)]
[(0.99999999117333271, 'banana'), (1.3137541292133003e-10, 'flower'), (1.075350457575297e-10, 0), (1.075350457575297e-10, 0)]
[(0.99999999999998435, 'frog'), (9.7983050064725333e-15, 'flower'), (8.9098551246838231e-17, 0), (8.9098551246838231e-17, 0)]
[(1.0, 'sugar'), (3.26514565412475e-20, 'frog'), (4.1929033382922643e-37, 0), (4.1929033382922643e-37, 0)]
[(0.99999999999999967, 'ladyfly'), (1.1857452360397176e-17, 'dog'), (5.9978040555265287e-18, 0), (5.9978040555265287e-18, 0)]
[(1.0, 'sugar'), (3.

[(0.99999999123022476, 'banana'), (1.3053682181706501e-10, 'flower'), (1.0684846609535153e-10, 0), (1.0684846609535153e-10, 0)]
[(1.0, 'flower'), (1.4849840027681496e-37, 0), (1.4849840027681496e-37, 0), (1.4849840027681496e-37, 0)]
[(0.99999999999938782, 'table'), (6.1865661159188443e-14, 'flower'), (6.8529808769855229e-15, 0), (6.8529808769855229e-15, 0)]
[(0.99999999999994238, 'malefly'), (1.8920710160986721e-14, 'dog'), (3.2290897020496581e-15, 'flower'), (4.3690035696108222e-16, 0)]
[(1.0, 'sweat'), (1.745823876652021e-36, 'flower'), (2.9227666599994022e-42, 0), (2.9227666599994022e-42, 0)]
[(1.0, 'lemon'), (1.8511510640751509e-38, 0), (1.8511510640751509e-38, 0), (1.8511510640751509e-38, 0)]
[(1.0, 'sugar'), (3.2798874472290189e-20, 'sweat'), (4.1928990455197288e-37, 0), (4.1928990455197288e-37, 0)]
[(0.99999999999998446, 'frog'), (9.7369357779640454e-15, 'flower'), (8.8540369517577315e-17, 0), (8.8540369517577315e-17, 0)]
[(1.0, 'sweat'), (1.7458238766513886e-36, 'flower'), (2.9

[(1.0, 'cow'), (2.869474356911031e-45, 0), (2.869474356911031e-45, 0), (2.869474356911031e-45, 0)]
[(1.0, 'chair'), (1.5065335729734201e-45, 'sugar'), (1.1834196636655601e-46, 'dog'), (6.1289441946265556e-48, 0)]
[(0.99999999125728189, 'banana'), (1.3013795822193656e-10, 'flower'), (1.0652190366994334e-10, 0), (1.0652190366994334e-10, 0)]
[(0.99999999999915645, 'shock'), (8.4227363531939447e-13, 'sugar'), (1.6388368164027061e-17, 0), (1.6388368164027061e-17, 0)]
[(0.99999999999999967, 'ladyfly'), (1.167281634233218e-17, 'dog'), (5.9504407620889352e-18, 0), (5.9504407620889352e-18, 0)]
[(1.0, 'cow'), (2.8694738337848277e-45, 0), (2.8694738337848277e-45, 0), (2.8694738337848277e-45, 0)]
[(1.0, 'cat'), (9.3977008506865884e-19, 'sugar'), (1.9811832705220601e-20, 0), (1.9811832705220601e-20, 0)]
[(1.0, 'chair'), (1.5065332487492097e-45, 'sugar'), (1.1834194066550863e-46, 'dog'), (6.1289428602588188e-48, 0)]
[(1.0, 'cow'), (2.8694738337848277e-45, 0), (2.8694738337848277e-45, 0), (2.86947383

[(1.0, 'dog'), (9.0222987979881961e-35, 'lemon'), (5.0727640655371758e-36, 0), (5.0727640655371758e-36, 0)]
[(0.99999999999999967, 'ladyfly'), (1.1601450323175122e-17, 'dog'), (5.93202837253434e-18, 0), (5.93202837253434e-18, 0)]
[(1.0, 'sugar'), (3.2955147453488587e-20, 'ladyfly'), (4.1928943468501117e-37, 0), (4.1928943468501117e-37, 0)]
[(1.0, 'chair'), (1.5011273821227912e-45, 'sugar'), (1.1750290618995767e-46, 'dog'), (6.1069452758038909e-48, 0)]
[(0.99999999999999967, 'ladyfly'), (1.1601450323174874e-17, 'dog'), (5.9320283725342091e-18, 0), (5.9320283725342091e-18, 0)]
[(1.0, 'cow'), (2.8594199545554981e-45, 0), (2.8594199545554981e-45, 0), (2.8594199545554981e-45, 0)]
[(0.99999999999992117, 'doodoo'), (9.7447186262199645e-16, 0), (9.7447186262199645e-16, 0), (9.7447186262199645e-16, 0)]
[(0.99999999999939171, 'table'), (6.1474134778796633e-14, 'flower'), (6.8095999003617622e-15, 0), (6.8095999003617622e-15, 0)]
[(0.9999999912898857, 'banana'), (1.2965727176862241e-10, 'flower'),

[(1.0, 'sugar'), (3.3038036361801189e-20, 'ladyfly'), (4.1928917750213486e-37, 0), (4.1928917750213486e-37, 0)]
[(0.99999999132127415, 'banana'), (1.2919448692388579e-10, 'flower'), (1.0574945114616101e-10, 0), (1.0574945114616101e-10, 0)]
[(1.0, 'flower'), (1.4849811279833078e-37, 0), (1.4849811279833078e-37, 0), (1.4849811279833078e-37, 0)]
[(0.99999999999998457, 'frog'), (9.6387605699094809e-15, 'flower'), (8.7647413998139133e-17, 0), (8.7647413998139133e-17, 0)]
[(0.99999999999992129, 'doodoo'), (9.7207372566321563e-16, 0), (9.7207372566321563e-16, 0), (9.7207372566321563e-16, 0)]
[(0.99999999999994327, 'malefly'), (1.8469744085412685e-14, 'dog'), (3.1899724777686982e-15, 'flower'), (4.3160641305037527e-16, 0)]
[(1.0, 'chair'), (1.4959260143339927e-45, 'sugar'), (1.166986931249623e-46, 'dog'), (6.085779679943607e-48, 0)]
[(1.0, 'flower'), (1.4849811279837329e-37, 0), (1.4849811279837329e-37, 0), (1.4849811279837329e-37, 0)]
[(0.99999999999851596, 'apple'), (1.849073749347586e-14, 0

In [ ]:
  # inference
    result = c.compute(recordNum=counter, patternNZ=activeColumnIndices,
               classification={"bucketIdx": bucketIdx, "actValue": record[0]},
                        learn=False, infer=True)

numRecords = 10
inputFile = open(_INPUT_FILE_PATH, "rb")
reader = csv.reader(inputFile)
# skip header rows
reader.next()
reader.next()
reader.next()


counter = 0
for record in reader:
   
    counter += 1
    if counter >= numRecords: break
            

    # Convert data string into Python date object.
    #dateString = datetime.datetime.strptime(record[1], "%m/%d/%y %H:%M")
    scentString = record[0]
    # Convert data value string into float.
    scent = categoryEncoder.encode(record[0])
    chem1 = float(record[1])
    chem2 = float(record[2])
    dopa = int(record[51])
    
    
    # Now we call the encoders to create bit representations for each value.
    #timeOfDayEncoder.encodeIntoArray(dateString, timeOfDayBits)
    #weekendEncoder.encodeIntoArray(dateString, weekendBits)
    scentBits = numpy.zeros(categoryEncoder.getWidth())
    chem1Bits = numpy.zeros(scalarEncoder.getWidth())
    chem2Bits = numpy.zeros(scalarEncoder.getWidth())
    
    
    categoryEncoder.encodeIntoArray(scentString,scentBits)
    scalarEncoder.encodeIntoArray(chem1, chem1Bits)
    scalarEncoder.encodeIntoArray(chem2, chem2Bits)
    
     # Concatenate all these encodings into one large encoding for Spatial
        # Pooling.
            
    encoding = numpy.concatenate(
        [scentBits, chem1Bits, chem2Bits])
    
    
    
    
    
    # Get the bucket info for this input value for classification.
    bucketIdx = categoryEncoder.getBucketIndices(scentString)[0]
    
    #active column indices 
        
   
    
    #use dopa value to learn scent n times in a row before learning next scent

            
    #for i in range(dopa):
        
        #dopacounter += 1
        
       
       # print(activeColumnIndices)
        
        
        #sp.compute(encoding, learn=True, activeArray=ArrayOfColumns)
        
        # learning
        #c.compute(recordNum= counter, patternNZ= activeColumnIndices,
               #classification={"bucketIdx":bucketIdx , "actValue": record[0]},
               #learn=True, infer=False)
            
        
        
       
        
        
        
       
    
        #Heatmap of columns 
        #array = numpy.reshape(activeDutyCycles, (45,45))
        
        
        #plt.imshow(array, cmap='hot', interpolation='nearest')
    
        #plt.show()
        
       
        
inputFile.close()
    
    
    
    
    
    
    
    
       